In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Download dataset 
nltk.download('movie_reviews')

# Load movie reviews dataset
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [3]:
# Convert to DataFrame
df = pd.DataFrame(documents, columns=['review', 'sentiment'])
df['review'] = df['review'].apply(lambda words: ' '.join(words))  # Convert list of words to string
df['sentiment'] = df['sentiment'].map({'neg': 0, 'pos': 1})  # Convert labels to numerical values

# Reduce dataset size to avoid memory issues
df = df.sample(n=2000, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Feature Extraction (Convert text to numerical format)
vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Option 1: Logistic Regression (generally performs well for text classification)
model = LogisticRegression(max_iter=2000, C=1.0)

#Train the selected model
model.fit(X_train_vect, y_train)

# Predictions
y_pred = model.predict(X_test_vect)

In [4]:
# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Examples of predictions
sample_reviews = [
    "This movie was fantastic! The story was engaging and the characters were well developed.",
    "I absolutely hated this film. It was boring and the acting was terrible.",
    "An average movie, some good parts but also many flaws.",
    "One of the best movies I have ever seen. Highly recommended!",
    "Worst movie ever. Do not waste your time."
]

# Transform the sample reviews using the trained vectorizer
sample_vect = vectorizer.transform(sample_reviews)

# Predict sentiment for the sample reviews
sample_preds = model.predict(sample_vect)

# Print results
for review, sentiment in zip(sample_reviews, sample_preds):
    print(f"Review: {review}\nPredicted Sentiment: {'Positive' if sentiment == 1 else 'Negative'}\n")

Accuracy: 0.85
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       197
           1       0.86      0.85      0.85       203

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

Confusion Matrix:
[[169  28]
 [ 31 172]]
Review: This movie was fantastic! The story was engaging and the characters were well developed.
Predicted Sentiment: Positive

Review: I absolutely hated this film. It was boring and the acting was terrible.
Predicted Sentiment: Negative

Review: An average movie, some good parts but also many flaws.
Predicted Sentiment: Positive

Review: One of the best movies I have ever seen. Highly recommended!
Predicted Sentiment: Positive

Review: Worst movie ever. Do not waste your time.
Predicted Sentiment: Negative

